In [11]:
import numpy as np


def compute_pagerank(G, alpha=0.85, q=None, max_iter=100, tol=1.0e-6):
    """
    Compute the PageRank of each node in the graph G using the power method.

    :param G: 2D numpy array representing the adjacency matrix of the graph
    :param alpha: Damping factor for the random walk
    :param q: Probability distribution for random jumps (uniform by default)
    :param max_iter: Maximum number of iterations for the power method
    :param tol: Tolerance for convergence
    :return: 1D numpy array containing the PageRank of each node
    """
    n = G.shape[0]  # Number of nodes

    # If q is not provided, use a uniform distribution
    if q is None:
        q = np.ones(n) / n

    # Initialize PageRank vector
    p = np.ones(n) / n

    # Normalize rows of G to represent probabilities
    row_sums = G.sum(axis=1)
    G_normalized = G / row_sums[:, np.newaxis]

    for _ in range(max_iter):
        p_new = alpha * G_normalized.T.dot(p) + (1 - alpha) * q

        # Check convergence
        if np.linalg.norm(p_new - p, ord=1) <= tol:
            break

        p = p_new

    return p


# Example graph (as an adjacency matrix)
# A simple graph with 4 nodes
G = np.array([
    [0, 1, 1, 0],
    [1, 0, 1, 1],
    [1, 1, 0, 1],
    [0, 1, 1, 0]
], dtype=float)

# Compute PageRank
pagerank = compute_pagerank(G)
pagerank

array([0.20478714, 0.29521286, 0.29521286, 0.20478714])

In [12]:
def compute_optimized_pagerank(G, alpha=0.85, q=None, max_iter=100, tol=1.0e-6,
                               incoming_link_weight=1.0, keyword_weight=1.0,
                               outgoing_link_weight=1.0, link_quality_weight=1.0):
    """
    Compute the optimized PageRank of each node in the graph G using the power method,
    considering additional factors like incoming links, keywords, outgoing links, and link quality.

    :param G: 2D numpy array representing the adjusted adjacency matrix of the graph with additional factors
    :param alpha: Damping factor for the random walk
    :param q: Probability distribution for random jumps (uniform by default)
    :param max_iter: Maximum number of iterations for the power method
    :param tol: Tolerance for convergence
    :param incoming_link_weight: Weight factor for incoming links
    :param keyword_weight: Weight factor for keyword relevance
    :param outgoing_link_weight: Weight factor for outgoing links
    :param link_quality_weight: Weight factor for link quality
    :return: 1D numpy array containing the PageRank of each node
    """
    n = G.shape[0]  # Number of nodes

    # If q is not provided, use a uniform distribution
    if q is None:
        q = np.ones(n) / n

    # Adjust weights in the adjacency matrix
    adjusted_G = G.copy()
    adjusted_G *= incoming_link_weight
    adjusted_G *= keyword_weight
    adjusted_G *= outgoing_link_weight
    adjusted_G *= link_quality_weight

    # Normalize rows of adjusted_G to represent probabilities
    row_sums = adjusted_G.sum(axis=1)
    G_normalized = adjusted_G / row_sums[:, np.newaxis]

    # Initialize PageRank vector
    p = np.ones(n) / n

    for _ in range(max_iter):
        p_new = alpha * G_normalized.T.dot(p) + (1 - alpha) * q

        # Check convergence
        if np.linalg.norm(p_new - p, ord=1) <= tol:
            break

        p = p_new

    return p


# Example graph with additional factors (simplified for demonstration)
# A simple graph with 4 nodes, weights are illustrative and not based on actual data
G = np.array([
    [0, 2, 3, 0],  # Incoming links, keywords, etc., for node 1
    [2, 0, 1, 1],  # Node 2
    [1, 2, 0, 1],  # Node 3
    [0, 1, 2, 0]   # Node 4
], dtype=float)

# Compute optimized PageRank
optimized_pagerank = compute_optimized_pagerank(G)
optimized_pagerank

array([0.22867099, 0.2944073 , 0.31081249, 0.16610923])

In [13]:
def compute_keyword_specific_pagerank(keyword_graphs, alpha=0.85, max_iter=100, tol=1.0e-6):
    """
    Compute the keyword-specific PageRank for each keyword graph.

    :param keyword_graphs: Dictionary of keyword graphs, where keys are keywords and values are adjacency matrices
    :param alpha: Damping factor for the random walk
    :param max_iter: Maximum number of iterations for the power method
    :param tol: Tolerance for convergence
    :return: Dictionary of PageRank vectors for each keyword
    """
    keyword_pageranks = {}
    for keyword, graph in keyword_graphs.items():
        # Uniform distribution for q
        q = np.ones(graph.shape[0]) / graph.shape[0]
        pagerank = compute_pagerank(graph, alpha, q, max_iter, tol)
        keyword_pageranks[keyword] = pagerank
    return keyword_pageranks


def aggregate_pageranks(keyword_pageranks, keyword_weights):
    """
    Aggregate the keyword-specific PageRank vectors into a final PageRank vector.

    :param keyword_pageranks: Dictionary of PageRank vectors for each keyword
    :param keyword_weights: Dictionary of weights for each keyword
    :return: Aggregated PageRank vector
    """
    aggregated_pagerank = np.zeros_like(list(keyword_pageranks.values())[0])
    for keyword, pagerank in keyword_pageranks.items():
        # Default weight is 1 if not specified
        weight = keyword_weights.get(keyword, 1)
        aggregated_pagerank += weight * pagerank
    aggregated_pagerank /= sum(keyword_weights.values())
    return aggregated_pagerank


# Example keyword-specific graphs (simplified for demonstration)
keyword_graphs = {
    "keyword1": np.array([
        [0, 2, 0, 0],
        [2, 0, 1, 0],
        [0, 1, 0, 1],
        [0, 0, 1, 0]
    ], dtype=float),
    "keyword2": np.array([
        [0, 1, 1, 0],
        [1, 0, 0, 1],
        [1, 0, 0, 1],
        [0, 1, 1, 0]
    ], dtype=float)
}

# Example keyword weights
keyword_weights = {
    "keyword1": 0.6,
    "keyword2": 0.4
}

# Compute keyword-specific PageRanks
keyword_pageranks = compute_keyword_specific_pagerank(keyword_graphs)

# Aggregate PageRanks
aggregated_pagerank = aggregate_pageranks(keyword_pageranks, keyword_weights)
aggregated_pagerank

array([0.24160266, 0.31018096, 0.25839734, 0.18981904])

In [14]:
def compute_advanced_pagerank(graphs, alpha=0.85, max_iter=100, tol=1.0e-6):
    """
    Compute the advanced PageRank considering different strategies: link addition/removal, link quality adjustment, and keyword adjustment.

    :param graphs: Dictionary of graphs for each strategy combination
    :param alpha: Damping factor for the random walk
    :param max_iter: Maximum number of iterations for the power method
    :param tol: Tolerance for convergence
    :return: Dictionary of PageRank vectors for each strategy
    """
    strategy_pageranks = {}
    for strategy, graph in graphs.items():
        # Uniform distribution for q
        q = np.ones(graph.shape[0]) / graph.shape[0]
        pagerank = compute_pagerank(graph, alpha, q, max_iter, tol)
        strategy_pageranks[strategy] = pagerank
    return strategy_pageranks


def evaluate_strategies(strategy_pageranks):
    """
    Evaluate and compare the strategies to find the Nash Equilibrium.

    :param strategy_pageranks: Dictionary of PageRank vectors for each strategy
    :return: Strategy that yields the highest PageRank, indicating Nash Equilibrium
    """
    best_strategy = None
    best_pagerank = None

    for strategy, pagerank in strategy_pageranks.items():
        if best_pagerank is None or np.max(pagerank) > np.max(best_pagerank):
            best_strategy = strategy
            best_pagerank = pagerank

    return best_strategy


# Example: Different strategy graphs (simplified for demonstration)
# Here, we consider different strategies as different graphs for simplicity
# 更新后的示例策略图
strategy_graphs = {
    "strategy1": np.array([
        [0, 2, 1, 0],
        [2, 0, 3, 1],
        [1, 3, 0, 2],
        [0, 1, 2, 0]
    ], dtype=float),
    "strategy2": np.array([
        [0, 1, 0, 1],
        [1, 0, 2, 0],
        [0, 2, 0, 3],
        [1, 0, 3, 0]
    ], dtype=float)
    # 可以根据需要添加更多策略和相应的图
}

# 重新计算每种策略的PageRanks
strategy_pageranks = compute_advanced_pagerank(strategy_graphs)

# 重新评估策略以找到Nash Equilibrium
nash_equilibrium_strategy = evaluate_strategies(strategy_pageranks)
nash_equilibrium_strategy, strategy_pageranks[nash_equilibrium_strategy]


# Compute PageRanks for each strategy
strategy_pageranks = compute_advanced_pagerank(strategy_graphs)

# Evaluate strategies to find Nash Equilibrium
nash_equilibrium_strategy = evaluate_strategies(strategy_pageranks)
nash_equilibrium_strategy

'strategy2'